In [2]:
# module import
import os
import sys
from datetime import datetime
import pickle

import numpy as np
import PIL.Image
import torch
import torchvision
                
sys.path.append('../cnn_preferred')
from utils import normalise_img, clip_extreme_pixel,get_cnn_features, img_deprocess, get_target_feature_shape
from activation_maximization import generate_preferred

In [3]:
## load network
#net = torchvision.models.resnet50(pretrained=True)
#net = torchvision.models.densenet121(pretrained=True)
net = torchvision.models.inception_v3(pretrained=True)
net.eval()

/Users/admin/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [4]:
## image mean and std for pre/de-process image for input network
img_mean=np.array([0.485, 0.456, 0.406],dtype=np.float),
img_std = np.array([0.229,0.224,0.225])

# if the model input is for 0-1 range, norm = 255, elif 0-255, norm = 1
norm = 255

In [5]:
## create save_dir
save_dir = '../result'
save_folder = 'jupyter_demo_torch_complexCNN_conv'
save_folder = save_folder + '_' + datetime.now().strftime('%Y%m%dT%H%M%S')
save_path = os.path.join(save_dir,save_folder)
os.makedirs(save_path, exist_ok=True)

In [6]:
# initial image for the optimization
## note that the input shape of inception_v3 model is (229,229,3), not (224,224,3)
h, w = 299,299
#h,w = 224, 224
initial_input = np.random.randint(0, 256, (h,w,3))

In [7]:
net

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [16]:
## set target layer 
#target_layer = "Conv2d_2a_3x3.conv"
target_layer = "fc"
target_layer = "Mixed_7a.branch7x7x3_3.conv"
target_layer_list = [target_layer]

In [12]:
# options
opts = {
    'img_mean': img_mean, #img_mean to preprocessing input image
    'img_std': img_std,   #img_std to preprocessing input image
    'norm': norm,         #if the model input is for 0-1 range, norm = 255, elif 0-255, norm = 1
    
    'iter_n': 200, # the total number of iterations for gradient descend

    'disp_every': 1, # display the information on the terminal for every n iterations

    'save_intermediate': True, # save the intermediate or not
    'save_intermediate_every': 10, # save the intermediate for every n iterations
    'save_intermediate_path': save_path, # the path to save the intermediate

    'lr_start':1, # learning rate
    'lr_end': 1,   # we can change learning rate linearly setteing these two parameters

    'momentum_start': 0.001, # gradient with momentum
    'momentum_end': 0.001,   # we can change momentum linearly setteing these two parameters too

    'decay_start': 0.001, # pixel decay for each iteration
    'decay_end': 0.001,   # we can also change pixel decay linealy 

    'image_blur': True, # Use image smoothing or not
    'sigma_start': 2.5, # the size of the gaussian filter for image smoothing
    'sigma_end': 0.5,

    'image_jitter': True, # use image jittering during optimization
    'jitter_size': 32,
    
    'use_p_norm_reg': False, # use p_norm regularization
    'p': 2,

    'use_TV_norm_reg': False,
    'TVbeta1': 1, 
    'TVbeta2':1.2,

    'clip_small_norm': True,
    'clip_small_norm_every': 1,
    'n_pct_start': 5,
    'n_pct_end': 5,

    'clip_small_contribution': True,
    'clip_small_contribution_every': 1,
    'c_pct_start': 5,
    'c_pct_end':5,
    'input_shape': (h,w,3) # you can set only input shape instead of initial_input image. in this case, intial_input  
                           # is 0-255 random image whose shape matches to input_shape
    }



In [13]:
channel_list = [14,56]

In [17]:
for channel in channel_list:
    #
    print('')
    print('channel='+str(channel))
    print('')

    # generate preferred image
    preferred_stim = generate_preferred(net, target_layer_list, channel, **opts)
    # save the results
    save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.npy'
    np.save(os.path.join(save_path,save_name), preferred_stim)

    save_name = 'preferred_img' + '_layer_' + str(target_layer) + '_channel_' + str(channel) + '.jpg'
    
    PIL.Image.fromarray(normalise_img(clip_extreme_pixel(preferred_stim, pct=0.04))).save(
                    os.path.join(save_path, save_name))


channel=14

iter=1; mean(abs(feat))=0.564463;
iter=2; mean(abs(feat))=0.0119259;
iter=3; mean(abs(feat))=0.0755982;
iter=4; mean(abs(feat))=0.109982;
iter=5; mean(abs(feat))=0.135763;
iter=6; mean(abs(feat))=0.0773831;
iter=7; mean(abs(feat))=0.103221;
iter=8; mean(abs(feat))=0.144326;
iter=9; mean(abs(feat))=0.150709;
iter=10; mean(abs(feat))=0.300074;
iter=11; mean(abs(feat))=0.255922;
iter=12; mean(abs(feat))=0.218662;
iter=13; mean(abs(feat))=0.226544;
iter=14; mean(abs(feat))=0.163751;
iter=15; mean(abs(feat))=0.174325;
iter=16; mean(abs(feat))=0.167798;
iter=17; mean(abs(feat))=0.179992;
iter=18; mean(abs(feat))=0.227519;
iter=19; mean(abs(feat))=0.229406;
iter=20; mean(abs(feat))=0.083505;
iter=21; mean(abs(feat))=0.208801;
iter=22; mean(abs(feat))=0.249036;
iter=23; mean(abs(feat))=0.102225;
iter=24; mean(abs(feat))=0.274501;
iter=25; mean(abs(feat))=0.39642;
iter=26; mean(abs(feat))=0.279395;
iter=27; mean(abs(feat))=0.227048;
iter=28; mean(abs(feat))=0.275613;
iter=29; mean(

iter=31; mean(abs(feat))=0.73636;
iter=32; mean(abs(feat))=0.338441;
iter=33; mean(abs(feat))=0.478486;
iter=34; mean(abs(feat))=0.800635;
iter=35; mean(abs(feat))=0.319914;
iter=36; mean(abs(feat))=0.246098;
iter=37; mean(abs(feat))=0.591715;
iter=38; mean(abs(feat))=0.394914;
iter=39; mean(abs(feat))=0.800919;
iter=40; mean(abs(feat))=0.535123;
iter=41; mean(abs(feat))=0.818757;
iter=42; mean(abs(feat))=0.290534;
iter=43; mean(abs(feat))=0.830389;
iter=44; mean(abs(feat))=1.18108;
iter=45; mean(abs(feat))=0.179454;
iter=46; mean(abs(feat))=0.62572;
iter=47; mean(abs(feat))=0.944852;
iter=48; mean(abs(feat))=0.854526;
iter=49; mean(abs(feat))=0.147346;
iter=50; mean(abs(feat))=0.468122;
iter=51; mean(abs(feat))=0.965901;
iter=52; mean(abs(feat))=0.450934;
iter=53; mean(abs(feat))=0.644899;
iter=54; mean(abs(feat))=0.750033;
iter=55; mean(abs(feat))=0.707614;
iter=56; mean(abs(feat))=0.841155;
iter=57; mean(abs(feat))=0.478211;
iter=58; mean(abs(feat))=1.01648;
iter=59; mean(abs(feat))